# 雲端載入資料

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#下載yolov5

In [2]:
!git clone https://github.com/ultralytics/yolov5.git

Cloning into 'yolov5'...
remote: Enumerating objects: 14106, done.
remote: Counting objects: 100% (124/124), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 14106 (delta 83), reused 105 (delta 69), pack-reused 13982
Receiving objects: 100% (14106/14106), 13.24 MiB | 25.25 MiB/s, done.
Resolving deltas: 100% (9737/9737), done.


#下載Data並放入指定路徑資料夾

In [3]:
!unzip -qq /content/drive/MyDrive/Training_Dataset_v3.zip
!unzip -qq /content/drive/MyDrive/Public_Testing_Dataset_v2.zip
!unzip -qq /content/drive/MyDrive/data.zip

In [4]:
import os
os.mkdir('/content/train/images')
os.mkdir('/content/train/labels')
os.mkdir('/content/val')
os.mkdir('/content/val/images')
os.mkdir('/content/val/labels')

#資料前處理-處理成yolov5對應之格式

In [5]:
import numpy as np
def retxt(path, height, width):
  if os.stat(path).st_size == 0:
    return
  data = open(path)
  list_of_lists = []
  for line in data:
    stripped_line = line.strip()
    ob_list = [np.float(s) for s in stripped_line.split(',')]
    ob_list[0] = np.int32(ob_list[0])
    ob_list[1] = (ob_list[1]+(ob_list[3]/2))/width
    ob_list[2] = (ob_list[2]+(ob_list[4]/2))/height
    ob_list[3] = ob_list[3]/width
    ob_list[4] = ob_list[4]/height
    list_of_lists.append(ob_list)
  np.savetxt(path, list_of_lists, fmt='%d %f %f %f %f', delimiter=' ', newline='\n')

In [6]:
import cv2

for i in range(1, 801, 1):
  if(i<10):
    img = "img000" + str(i) + ".png"
    text = "img000" + str(i) + ".txt"
  elif(i<100):
    img = "img00" + str(i) + ".png"
    text = "img00" + str(i) + ".txt"
  else:
    img = "img0" + str(i) + ".png"
    text = "img0" + str(i) + ".txt"
  height, width, _ = cv2.imread('/content/train/' + str(img)).shape
  retxt('/content/train/' + str(text), height, width)
  os.replace('/content/train/' + str(img), '/content/train/images/' + str(img))
  os.replace('/content/train/' + str(text), '/content/train/labels/' + str(text))
for i in range(801, 1001, 1):
  if(i<1000):
    img = "img0" + str(i) + ".png"
    text = "img0" + str(i) + ".txt"
  else:
    img = "img" + str(i) + ".png"
    text = "img" + str(i) + ".txt"
  height, width, _ = cv2.imread('/content/train/' + str(img)).shape
  retxt('/content/train/' + str(text), height, width)
  os.replace('/content/train/' + str(img), '/content/val/images/' + str(img))
  os.replace('/content/train/' + str(text), '/content/val/labels/' + str(text))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if __name__ == '__main__':


#yolov5x模型訓練

##yolov5x yaml檔之nc要改為class數量

In [21]:
!python /content/yolov5/train.py --data ./data.yaml --cfg /content/yolov5/models/yolov5x.yaml --weights '/content/yolov5/runs/train/exp2/weights/last.pt' --batch-size -1 --epochs 60 --rect --imgsz 960 --image-weights

train: weights=/content/yolov5/runs/train/exp2/weights/last.pt, cfg=/content/yolov5/models/yolov5x.yaml, data=./data.yaml, hyp=yolov5/data/hyps/hyp.scratch-low.yaml, epochs=60, batch_size=-1, imgsz=960, rect=True, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=True, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5/runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.2-195-gdf80e7c Python-3.7.14 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, ob

In [24]:
os.replace('/content/yolov5/runs/train/exp3/weights/best.pt', '/content/drive/MyDrive/yolov5')
os.replace('/content/yolov5/runs/train/exp3/weights/last.pt', '/content/drive/MyDrive/yolov5')

OSError: ignored

#測試model(單張圖)

In [28]:
import torch
model = torch.hub.load('ultralytics/yolov5', 'custom' , path='/content/yolov5/runs/train/exp3/weights/best.pt')

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2022-10-16 Python-3.7.14 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
YOLOv5x summary: 322 layers, 86193601 parameters, 0 gradients, 203.8 GFLOPs
Adding AutoShape... 


In [29]:
result = model('/content/public/img1022.png')

In [30]:
result #yolov5x best epoch 180

YOLOv5 <class 'models.common.Detections'> instance
image 1/1: 1080x1920 22 cars, 1 hov, 13 motorcycles
Speed: 63.3ms pre-process, 61.5ms inference, 1.6ms NMS per image at shape (1, 3, 384, 640)

In [27]:
result #yolov5x last epoch 180

YOLOv5 <class 'models.common.Detections'> instance
image 1/1: 1080x1920 22 cars, 2 hovs, 13 motorcycles
Speed: 68.4ms pre-process, 64.3ms inference, 1.9ms NMS per image at shape (1, 3, 384, 640)

In [13]:
result #yolov5x best epoch 120

YOLOv5 <class 'models.common.Detections'> instance
image 1/1: 1080x1920 23 cars, 11 motorcycles
Speed: 93.5ms pre-process, 62.1ms inference, 5.5ms NMS per image at shape (1, 3, 384, 640)

In [16]:
result #yolov5x last epoch 120

YOLOv5 <class 'models.common.Detections'> instance
image 1/1: 1080x1920 23 cars, 11 motorcycles
Speed: 61.9ms pre-process, 61.9ms inference, 2.5ms NMS per image at shape (1, 3, 384, 640)

In [24]:
result #yolov5x best epoch 60

YOLOv5 <class 'models.common.Detections'> instance
image 1/1: 1080x1920 25 cars, 1 hov, 6 motorcycles
Speed: 70.7ms pre-process, 69.0ms inference, 4.4ms NMS per image at shape (1, 3, 384, 640)

In [16]:
result #yolov5x last epoch 60

YOLOv5 <class 'models.common.Detections'> instance
image 1/1: 1080x1920 24 cars, 1 hov, 8 motorcycles
Speed: 73.1ms pre-process, 63.9ms inference, 5.2ms NMS per image at shape (1, 3, 384, 640)

In [17]:
result.save()

Saved 1 image to runs/detect/exp


In [13]:
result #yolov5m epoch 200

YOLOv5 <class 'models.common.Detections'> instance
image 1/1: 1080x1920 22 cars, 3 hovs, 10 motorcycles
Speed: 99.3ms pre-process, 24.5ms inference, 2.2ms NMS per image at shape (1, 3, 384, 640)

#testing data預測

In [31]:
!python /content/yolov5/detect.py --weights /content/yolov5/runs/train/exp3/weights/best.pt --img 1080 --conf 0.25 --save-txt --source /content/public

detect: weights=['/content/yolov5/runs/train/exp3/weights/best.pt'], source=/content/public, data=yolov5/data/coco128.yaml, imgsz=[1080, 1080], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v6.2-195-gdf80e7c Python-3.7.14 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
YOLOv5x summary: 322 layers, 86193601 parameters, 0 gradients, 203.8 GFLOPs
WARNING ⚠️ --img-size [1080, 1080] must be multiple of max stride 32, updating to [1088, 1088]
image 1/500 /content/public/img1001.png: 640x1088 36 cars, 2 hovs, 1 person, 3 motorcycles, 98.8ms
image 2/500 /content/public/img1002.png: 640x1088 20 cars, 4 hovs, 1 person, 24 motorcycles, 93.5ms
image 3/500 /content

#預測目標之csv

In [32]:
import csv
from os import listdir

with open('output.csv', 'w', newline='') as csvfile:
  writer = csv.writer(csvfile)
  for text in listdir('/content/yolov5/runs/detect/exp2/labels/'):
    height, width, _ = cv2.imread('/content/yolov5/runs/detect/exp2/' + text.replace('.txt','') + '.png').shape
    data = open('/content/yolov5/runs/detect/exp2/labels/' + text.replace('.txt','') + '.txt')
    for line in data:
      stripped_line = line.strip()
      ob_list = [np.float(s) for s in stripped_line.split(' ')]
      ob_list[1] = (ob_list[1]-(ob_list[3]/2))*width
      ob_list[2] = (ob_list[2]-(ob_list[4]/2))*height
      ob_list[3] = ob_list[3]*width
      ob_list[4] = ob_list[4]*height
      writer.writerow([text.replace('.txt',''), np.int32(ob_list[0]), np.int32(ob_list[1]), np.int32(ob_list[2]), np.int32(ob_list[3]), np.int32(ob_list[4])])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  # This is added back by InteractiveShellApp.init_path()
